In [1]:
import math
import numpy as np
from numba import jit, njit, prange
from joblib import Parallel, delayed
from functools import partial
import multiprocessing as mp
from multiprocessing import Pool, Value, Array
from ipyparallel import Client
import pandas as pd
import timeit
%load_ext cython

Questions for Bo:

- Issues with parallelization
    - Can show that for serial cython version, my function fails when I use `with cython.nogil:`
    - Parallel numba code kills my kernel
- Do cython functions not need to have an explicit return?
- Why does cython not accept bool as a data type?

**1**. (100 points)

Write a predicate function `is_prime` that efficiently checks whether a number is prime. Use this to write a second function `primes_between` that returns the prime numbers between two integers as a `numpy` array.

- (10 points) Do this in regular Python 
- (10 points) Accelerate using `numba` (serial version) 
- (15 points) Accelerate using `numba` (parallel version)
- (10 points) Accelerate using `cython` (serial version) 
- (15 points) Accelerate using `cython` (parallel version)
- (10 points) Report the speed-up multiplier as an integer of the `numba` and `cython` serial and parallel versions using `timeit` in a DataFrame for the numbers between 0 and 1,000,000
- (10 points each) Run the serial version of the python `primes_between` function in parallel using
    - `multiprocessing`
    - `joblib`
    - `ipyparallel`

- (10 points) Do this in regular Python 



*Solution from Midterm 1 is used for is_prime( ) function*

In [2]:
def is_prime(n):
    """Returns True if a given integer n is prime and false otherwise"""

    if n == 2:
        return True
    elif n < 2 or n % 2 == 0:
        return False
    else:
        for i in range(3, int(np.sqrt(n))+1, 2):
            if n % i == 0:
                return False
    return True

In [3]:
def primes_between(n1, n2):
    """Returns prime numbers between n1 and n2 (inclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Initialize output and loop through all numbers between n1 and n2
    primes = []
    for num in range(n1, n2 + 1):
        if is_prime(num):
            primes.append(num)
    
    # Return result
    return np.array(primes)

In [18]:
def primes_between(n1, n2):
    """Returns prime numbers between n1 and n2 (inclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Use boolean indexing to figure out which numbers in desired range are prime
    prime_bool = [False for i in range(n1, n2 + 1)]
    for index, num in enumerate(range(n1, n2 + 1)):
        if is_prime(num):
            prime_bool[index] = True
    
    # Return prime numbers as np array
    return np.array([val for index, val in enumerate(range(n1, n2 + 1)) if prime_bool[index]])

In [44]:
%%timeit
#n1 = 1
#n2 = 5
#[False for i in range(n1, n2 + 1)]
[False] * (n2 - n1 + 1)

183 ns ± 0.537 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [45]:
def primes_between(n1, n2):
    """Returns prime numbers between n1 and n2 (inclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Use boolean indexing to figure out which numbers in desired range are prime
    prime_bool = [False] * (n2 - n1 + 1)
    index = 0
    for num in range(n1, n2 + 1):
        if is_prime(num):
            prime_bool[index] = True
        index += 1
    
    # Return prime numbers as np array
    return np.array([val for index, val in enumerate(range(n1, n2 + 1)) if prime_bool[index]])

In [46]:
# Prove that function works
primes_between(0, 50)

array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47])

In [47]:
%%timeit
primes_between(0, 1000)

1.53 ms ± 7.46 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [21]:
@jit(nopython=True, cache=True)
def is_prime_numba_serial(n):
    """Returns True if a given integer n is prime and false otherwise"""

    if n == 2:
        return True
    elif n < 2 or n % 2 == 0:
        return False
    else:
        for i in range(3, int(np.sqrt(n))+1, 2):
            if n % i == 0:
                return False
    return True

In [24]:
@jit(nopython=True, cache=True)
def primes_between_numba_serial(n1, n2):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Use boolean indexing to figure out which numbers in desired range are prime
    prime_bool = [False for i in range(n1, n2 + 1)]
    for index, num in enumerate(range(n1, n2 + 1)):
        if is_prime_numba_serial(num):
            prime_bool[index] = True
    
    # Return prime numbers as np array
    return np.array([val for index, val in enumerate(range(n1, n2 + 1)) if prime_bool[index]])

In [25]:
# Prove that function works
primes_between_numba_serial(0, 50)

array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47])

In [26]:
%%timeit
primes_between_numba_serial(0, 1000)

56.6 µs ± 420 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


- (15 points) Accelerate using `numba` (parallel version)



In [97]:
@njit(parallel = True)
def is_prime_numba_parallel(n):
    """Returns True if a given integer n is prime and false otherwise"""

    if n == 2:
        return True
    elif n < 2 or n % 2 == 0:
        return False
    else:
        for i in prange(3, int(np.sqrt(n))+1, 2):
            if n % i == 0:
                return False
    return True

In [66]:
@njit(parallel = True)
def primes_between_numba_parallel(n1, n2):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Use boolean indexing to figure out which numbers in desired range are prime
    prime_bool = [False] * (n2 - n1 + 1)
    index = 0
    for num in prange(n1, n2 + 1):
        prime_bool[index] = is_prime_numba_parallel(num)
        index += 1
    
    # Return prime numbers as np array
    return np.array([val for index, val in enumerate(range(n1, n2 + 1)) if prime_bool[index]])

In [68]:
np.arange(1, 3)

array([1, 2])

In [78]:
primes = np.arange(26)

In [79]:
bool_vec = [True, False] * 13

In [80]:
primes[bool_vec]

array([ 0,  2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24])

In [83]:
def primes_between(n1, n2):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
        
    # Initialize numpy array and boolean vector
    primes = np.arange(n1, n2 + 1)
    prime_bool = [False] * (n2 - n1 + 1)
    
    for index, num in enumerate(primes):
        prime_bool[index] = is_prime(num)
    
    return primes[prime_bool]

In [84]:
primes_between(0, 10)

array([2, 3, 5, 7])

In [81]:
@njit(parallel = True)
def primes_between_numba_parallel(n1, n2):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
        
    # Initialize numpy array and boolean vector
    primes = np.arange(n1, n2 + 1)
    prime_bool = [False] * (n2 - n1 + 1)
    
    for index, num in enumerate(primes):
        prime_bool[index] = is_prime_numba_parallel(num)
    
    return primes[prime_bool]

In [86]:
primes.shape[0]

26

In [100]:
{num: False for num in range(n1, n2 + 1)}

{1: False, 2: False, 3: False, 4: False, 5: False}

In [98]:
@njit(parallel = True)
def primes_between_numba_parallel(n1, n2):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
        
    # Initialize numpy array and boolean vector
    primes = np.arange(n1, n2 + 1)
    prime_bool = [False] * (n2 - n1 + 1)
    
    for val in prange(primes.shape[0]):
        prime_bool[val] = is_prime_numba_parallel(primes[val])
    
    return primes[prime_bool]

In [99]:
primes_between_numba_parallel(0, 50)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of Function(<built-in function getitem>) with argument(s) of type(s): (array(int64, 1d, C), list(bool))
 * parameterized
In definition 0:
    All templates rejected with literals.
In definition 1:
    All templates rejected without literals.
In definition 2:
    All templates rejected with literals.
In definition 3:
    All templates rejected without literals.
In definition 4:
    All templates rejected with literals.
In definition 5:
    All templates rejected without literals.
In definition 6:
    All templates rejected with literals.
In definition 7:
    All templates rejected without literals.
In definition 8:
    All templates rejected with literals.
In definition 9:
    All templates rejected without literals.
In definition 10:
    All templates rejected with literals.
In definition 11:
    All templates rejected without literals.
In definition 12:
    TypeError: unsupported array index type list(

In [67]:
%%timeit
primes_between_numba_parallel(0, 1000)

36.6 µs ± 7.06 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


/opt/conda/lib/python3.6/site-packages/numba/typed_passes.py:293: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "<ipython-input-66-3b98cfa8ac5c>", line 2:
@njit(parallel = True)
def primes_between_numba_parallel(n1, n2):
^

  state.func_ir.loc))


In [63]:
primes_between_numba_parallel.parallel_diagnostics(level=4)

 
 Parallel Accelerator Optimizing:  Function primes_between_numba_parallel, 
<ipython-input-61-3b98cfa8ac5c> (1)  


Parallel loop listing for  Function primes_between_numba_parallel, <ipython-input-61-3b98cfa8ac5c> (1) 
--------------------------------------------------------------------------------------------------|loop #ID
@njit(parallel = True)                                                                            | 
def primes_between_numba_parallel(n1, n2):                                                        | 
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""       | 
                                                                                                  | 
    # Check to see which argument is larger                                                       | 
    if n1 > n2:                                                                                   | 
        n1, n2 = n2, n1                                         

- (10 points) Accelerate using `cython` (serial version) 



In [8]:
%%cython

import cython
import numpy as np

# Primitive function
cdef is_prime_cython_serial(int n):
    """Returns True if a given integer n is prime and false otherwise"""
    
    cdef int i
    if n == 2:
        return True
    elif n < 2 or n % 2 == 0:
        return False
    else:
        for i in range(3, int(np.sqrt(n)) + 1, 2):
            if n % i == 0:
                return False
    return True

# 
def primes_between_cython_serial(int n1, int n2, list primes):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Loop through all numbers between n1 and n2
    cdef int num
    for num in range(n1, n2 + 1):
        if is_prime_cython_serial(num):
            primes.append(num)
    
    # Return result
    return np.array(primes)

In [9]:
# Prove that function works
primes_between_cython_serial(0, 50, [])

array([ 2,  3,  5,  7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47])

- (15 points) Accelerate using `cython` (parallel version)



- (10 points) Report the speed-up multiplier as an integer of the `numba` and `cython` serial and parallel versions using `timeit` in a DataFrame for the numbers between 0 and 1,000,000



*Values in data frame are calculated as (time for serial version in python) / (time for sped up version). A larger integer value denotes a greater speed-up*

In [10]:
# Create dataframe
df = pd.DataFrame(dict(Serial = [0, 0], Parallel = [0, 0]), index = ['numba', 'cython'])

# Set upper bound
upper = 1000000

# Calculate reference time
ref = %timeit -o -r3 -n3 primes_between(0, upper)
ref = ref.average

# numba serial
numba_serial = %timeit -o -r3 -n3 primes_between_numba_serial(0, upper)
numba_serial = numba_serial.average
df.iloc[0, 0] = int(np.round(ref / numba_serial))

# numba parallel
#numba_parallel = %timeit -o -r3 -n3 primes_between_numba_parallel(0, upper)
#numba_parallel = numba_parallel.average
#df.iloc[0, 1] = int(np.round(ref / numba_parallel))

# cython serial
cython_serial = %timeit -o -r3 -n3 primes_between_cython_serial(0, upper, [])
cython_serial = cython_serial.average
df.iloc[1, 0] = int(np.round(ref / cython_serial))

# cython parallel
#cython_parallel = %timeit -o -r3 -n3 primes_between_cython_parallel(0, upper)
#cython_parallel = cython_parallel.average
#df.iloc[1, 1] = int(np.round(ref / cython_parallel))

df

3.33 s ± 7.63 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)
216 ms ± 2.53 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)
858 ms ± 3.42 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


,Serial,Parallel
numba,15,0
cython,4,0


- (10 points) `multiprocessing`



In [11]:
# Create array with appropriate partitions
loop_array = np.array([0, 125000, 125001, 250000, 250001, 375000, 375001, 500000, 
                       500001, 625000, 625001, 750000, 750001, 875000, 875001, 1000000])

# Parallelize using multiprocessing
with mp.Pool(processes = 8) as pool:
    res = pool.starmap(primes_between, np.array_split(loop_array, 8))
    
# Show result
np.concatenate(res)

array([     2,      3,      5, ..., 999961, 999979, 999983])

- (10 points) `joblib`



In [12]:
res = Parallel(n_jobs=8)(delayed(primes_between)(max(0, i - 124999), i) for i in range(0, 1000001, 125000))
np.concatenate(res).astype('int')

array([     2,      3,      5, ..., 999961, 999979, 999983])

- (10 points) `ipyparallel`

In [13]:
# Connect to cluster of remote engines
rc = Client()
dv = rc[:]

In [14]:
# Define nested primes_between function
def primes_between_ipyparallel(n1, n2):
    """Returns prime numbers between n1 and n2 (exclusive on both ends) as a numpy array"""
    
    # Import numpy
    import numpy as np
    
    # Define predicate function with "primes_between" so that it is recognized within cluser
    def is_prime_ipyparallel(n):
        """Returns True if a given integer n is prime and false otherwise"""

        # Function
        if n == 2:
            return True
        elif n < 2 or n % 2 == 0:
            return False
        else:
            for i in range(3, int(np.sqrt(n))+1, 2):
                if n % i == 0:
                    return False
        return True
    
    # Check to see which argument is larger
    if n1 > n2:
        n1, n2 = n2, n1
    
    # Initialize output and loop through all numbers between n1 and n2
    primes = []
    for num in range(n1, n2 + 1):
        if is_prime_ipyparallel(num):
            primes.append(num)
    
    # Return result
    return np.array(primes)

In [15]:
res = dv.map_sync(primes_between_ipyparallel, [0, 250001, 500001, 750001], [250000, 500000, 750000, 1000000])
np.concatenate(res)

array([     2,      3,      5, ..., 999961, 999979, 999983])